# 載入所需套件

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from urllib.request import urlretrieve
import statsmodels.api as sm
import zipfile

# 定義抓取Fama三因子資料函數

In [2]:
def get_fama_french():
    ff_url='https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip'
    #根據連結下載壓縮檔並重新命名
    urlretrieve(ff_url,'fama_french.zip')

    #提取壓縮檔裡面的檔案
    zip_file=zipfile.ZipFile('fama_french.zip','r')
    zip_file.extractall()
    zip_file.close()

    ff_factors=pd.read_csv('F-F_Research_Data_Factors.csv',skiprows=3,index_col=0)
    #只要上方表格，以全列為空白做為區隔
    ff_row=ff_factors.isnull().any(1).to_numpy().nonzero()[0][0]
    ff_factors=pd.read_csv('F-F_Research_Data_Factors.csv', skiprows=3,nrows=ff_row,index_col=0)
    ff_factors.index=pd.to_datetime(ff_factors.index,format='%Y%m')
    #改成每月最後一日格式:Ex.2016-09-30
    ff_factors.index=ff_factors.index+pd.offsets.MonthEnd()
    ff_factors=ff_factors.apply(lambda x:x/ 100)
    
    return ff_factors

# 定義計算月報酬率變化函數

In [3]:
def get_return_data(price_data,period='M'):
    price=price_data.resample(period).last()
    return_data=pd.DataFrame(price.pct_change()[1:])
    return_data.columns=['portfolio']
    
    return return_data

# 定義跑Fama三因子迴歸函數

In [4]:
def run_reg_model(ticker,start,end):
    ff_data=get_fama_french()
    ff_last=ff_data.index[-1].date()
    
    price_data=pdr.DataReader(ticker,'yahoo',start,end)['Adj Close'].loc[:ff_last]
    return_data=get_return_data(price_data,'M')
    
    all_data=pd.merge(return_data,ff_data,how='inner',left_index= True,right_index=True)
    all_data.rename(columns={'Mkt-RF':'mkt_excess'},inplace=True)
    all_data['port_excess']=all_data['portfolio']-all_data['RF']
    
    model=sm.formula.ols(formula='port_excess ~ mkt_excess + SMB + HML',data=all_data).fit()
    model_params=model.params
    model_summary=model.summary()
    
    return model_params,model_summary

# 實際跑Fama三因子迴歸

In [5]:
ggrax_params,ggrax_summary=run_reg_model('GGRAX',start='1999-05-01',end='2020-06-28')
print(ggrax_params)

Intercept    -0.000076
mkt_excess    1.025940
SMB          -0.188985
HML          -0.176255
dtype: float64


In [6]:
ggrax_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            port_excess   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     1182.
Date:                Mon, 06 Jul 2020   Prob (F-statistic):          1.61e-146
Time:                        09:49:54   Log-Likelihood:                 757.56
No. Observations:                 252   AIC:                            -1507.
Df Residuals:                     248   BIC:                            -1493.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -7.572e-05      0.001     -0.099      0.921      -0.002       0.001
mkt_excess     1.0259      0.018     58.446      0.000       0.991       1.061
SMB           -0.1890      0.025     -7.697      0.000      -0.237      -0.141
HML           -0.1763      0.023     -7.538      0.000      -0.222      -0.130
==============================================================================
Omnibus:                       17.938   Durbin-Watson:                   1.542
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.802
Skew:                           0.093   Prob(JB):                     7.63e-13
Kurtosis:                       5.298   Cond. No.                         35.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
fcntx_params,fcntx_summary=run_reg_model('FCNTX',start='1999-05-01',end='2020-06-28')
print(fcntx_params)

Intercept     0.002792
mkt_excess    0.805479
SMB           0.031234
HML          -0.074881
dtype: float64


In [8]:
fcntx_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            port_excess   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     434.7
Date:                Mon, 06 Jul 2020   Prob (F-statistic):           1.21e-98
Time:                        09:49:56   Log-Likelihood:                 686.56
No. Observations:                 253   AIC:                            -1365.
Df Residuals:                     249   BIC:                            -1351.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0028      0.001      2.726      0.007       0.001       0.005
mkt_excess     0.8055      0.023     34.316      0.000       0.759       0.852
SMB            0.0312      0.033      0.953      0.341      -0.033       0.096
HML           -0.0749      0.031     -2.395      0.017      -0.136      -0.013
==============================================================================
Omnibus:                       33.655   Durbin-Watson:                   2.185
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              131.612
Skew:                          -0.432   Prob(JB):                     2.63e-29
Kurtosis:                       6.426   Cond. No.                         35.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""